In [50]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.preprocessing import   LabelEncoder
from sklearn.linear_model import LogisticRegression

In [9]:
df=pd.read_csv('data/Results_TTM.csv',sep=";",header=0)
df=df.drop(columns=["Step status","Current trend","Torque rate min","Torque rate max","Torque rate trend","CVILOGIX","Identifier6","Identifier7","Identifier8","Identifier9","Identifier10","Second transducer torque deviation","Second transducer angle deviation","Result type","Pulse counter","Angle offset","AO torque rate"])

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 19 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Result status  400000 non-null  object 
 1   Result number  400000 non-null  int64  
 2   Time result    400000 non-null  object 
 3   Pset ID        399901 non-null  float64
 4   Step ID        399901 non-null  float64
 5   Error Code     22688 non-null   object 
 6   Torque min     399901 non-null  float64
 7   Torque         400000 non-null  float64
 8   Torque max     399901 non-null  float64
 9   Angle min      399901 non-null  float64
 10  Angle          400000 non-null  float64
 11  Angle max      399901 non-null  float64
 12  Pset name      399901 non-null  object 
 13  VIN            172013 non-null  object 
 14  Identifier1    172013 non-null  object 
 15  Identifier2    170608 non-null  object 
 16  Identifier3    168406 non-null  object 
 17  Identifier4    13920 non-null

In [13]:
df=df[['Result status','Result number','Time result','Pset ID','Step ID','Error Code', 'Torque min','Torque','Torque max','Angle min','Angle','Angle max','Pset name','VIN','Identifier1','Identifier2','Identifier3','Identifier4','Identifier5']]

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 19 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Result_status  400000 non-null  object 
 1   Result_number  400000 non-null  int64  
 2   Time_result    400000 non-null  object 
 3   Pset_ID        399901 non-null  float64
 4   Step_ID        399901 non-null  float64
 5   Error Code     22688 non-null   object 
 6   Torque_min     399901 non-null  float64
 7   Torque         400000 non-null  float64
 8   Torque_max     399901 non-null  float64
 9   Angle_min      399901 non-null  float64
 10  Angle          400000 non-null  float64
 11  Angle_max      399901 non-null  float64
 12  Pset_name      399901 non-null  object 
 13  VIN            172013 non-null  object 
 14  Identifier1    172013 non-null  object 
 15  Identifier2    170608 non-null  object 
 16  Identifier3    168406 non-null  object 
 17  Identifier4    13920 non-null

In [17]:
df=df.rename(columns={"Result status":"Result_status","Result number":"Result_number","Pset ID":"Pset_ID","Step ID":"Step_ID","Torque min":"Torque_min","Torque max":"Torque_max","Angle min":"Angle_min","Angle max":"Angle_max","Pset name":"Pset_name",'Error code':'Error_code','Result_status':'Result_status','Time result':'Time_result'})


In [ ]:
df = df[~df["Pset_name"].str.contains("Poka",na=False)]

In [18]:
df.astype({'Time_result': 'datetime64[ns, US/Eastern]'}).dtypes()

TypeError: 'Series' object is not callable

In [20]:
df['Resultbin']=df['Result_status']=='OK'

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 398943 entries, 0 to 399999
Data columns (total 20 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Result_status  398943 non-null  object 
 1   Result_number  398943 non-null  int64  
 2   Time_result    398943 non-null  object 
 3   Pset_ID        398844 non-null  float64
 4   Step_ID        398844 non-null  float64
 5   Error Code     21631 non-null   object 
 6   Torque_min     398844 non-null  float64
 7   Torque         398943 non-null  float64
 8   Torque_max     398844 non-null  float64
 9   Angle_min      398844 non-null  float64
 10  Angle          398943 non-null  float64
 11  Angle_max      398844 non-null  float64
 12  Pset_name      398844 non-null  object 
 13  VIN            172013 non-null  object 
 14  Identifier1    172013 non-null  object 
 15  Identifier2    170608 non-null  object 
 16  Identifier3    168406 non-null  object 
 17  Identifier4    13920 non-null

In [44]:
dfX=df.drop(columns={'Result_status','Result_number','Time_result','Pset_ID','Identifier4','Identifier5','Identifier2','Error Code','Torque','Angle'})

In [67]:
dfX=dfX.dropna()
dfX = dfX[pd.to_numeric(dfX['Identifier3'],errors='coerce').notna()]
dfX.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 168002 entries, 10 to 399996
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Step_ID        168002 non-null  float64
 1   Torque_min     168002 non-null  float64
 2   Torque_max     168002 non-null  float64
 3   Angle_min      168002 non-null  float64
 4   Angle_max      168002 non-null  float64
 5   Pset_name      168002 non-null  object 
 6   VIN            168002 non-null  object 
 7   Identifier1    168002 non-null  object 
 8   Identifier3    168002 non-null  object 
 9   Resultbin      168002 non-null  bool   
 10  Pset_name_cat  168002 non-null  int32  
 11  Modelo         168002 non-null  int32  
dtypes: bool(1), float64(5), int32(2), object(4)
memory usage: 11.7+ MB


In [68]:
enc=LabelEncoder()
dfX['Identifier3'].astype('float')
dfX['Pset_name_cat'] = enc.fit_transform(dfX['Pset_name'])
dfX['Modelo'] = enc.fit_transform(dfX['Identifier1'])
dfX['Identifier3_cat'] = enc.fit_transform(dfX['Identifier3'])

TypeError: Encoders require their input to be uniformly strings or numbers. Got ['float', 'str']

In [47]:
OKS = len(df[df["Result_status"].str.contains("OK",na=False)])
NOKS=len(df[df["Result_status"].str.contains("NOK",na=False)])
NOKS/OKS

0.05391998355654818

In [70]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=3, shuffle=True)
X=dfX.drop(columns={'Resultbin','Pset_name','VIN','Identifier1'}).values
y=dfX[['Resultbin']]
list(kf.split(X))

[(array([     0,      3,      4, ..., 167998, 167999, 168001]),
  array([     1,      2,      6, ..., 167977, 167982, 168000])),
 (array([     0,      1,      2, ..., 167997, 168000, 168001]),
  array([     3,     10,     11, ..., 167994, 167998, 167999])),
 (array([     1,      2,      3, ..., 167998, 167999, 168000]),
  array([     0,      4,      5, ..., 167996, 167997, 168001]))]

In [54]:
for train,test in kf.split(X):
    print (train,test)

[     0      2      5 ... 168014 168019 168020] [     1      3      4 ... 168016 168017 168018]
[     0      1      2 ... 168018 168019 168020] [     9     10     11 ... 168011 168012 168014]
[     1      3      4 ... 168016 168017 168018] [     0      2      5 ... 168013 168019 168020]


In [71]:
model = LogisticRegression()
model.fit(X, y)
(model.score(X_test, y_test))
print(scores)

C:\Users\guill\AppData\Roaming\Python\Python39\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\guill\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


NameError: name 'y_test' is not defined